In [1]:
import cv2
import time
import threading
import numpy as np
import keyboard
from tensorflow.keras.models import load_model
import time

In [2]:
A = './model/basic_model-best-model-2020-10-11_16-41-12.h5'
B = './model/basic_model-best-model-2020-10-24_21-50-57.h5'

C = './model/basic_model-best-model-2020-10-27_07-21-28.h5'
D = './model/basic_model-best-model-2020-10-27_16-19-10.h5'

In [3]:
def KBC(ac):
    pred = int(ac)
    KAC = ['a','d','s','w','[',']']
    # KAC = ['left','right','down','up',']','[']
    keyboard.send(KAC[pred],do_release=False)
    time.sleep(0.25)
    keyboard.release(KAC[pred])

In [4]:
class ipcamCapture:
    def __init__(self, URL):
        self.Frame = []
        self.Framecube = []	
        self.img = []
        self.status = False
        self.isstop = False
        self.x = 0
        self.flag = False
        	
	# 攝影機連接。
        self.capture = cv2.VideoCapture(URL)

    def start(self):
	# 把程式放進子執行緒，daemon=True 表示該執行緒會隨著主執行緒關閉而關閉。
        print('ipcam started!')
        threading.Thread(target=self.queryframe, daemon=True, args=()).start()

    def stop(self):
	# 記得要設計停止無限迴圈的開關。
        self.isstop = True
        print('ipcam stopped!')
   
    def getframe(self):
	# 當有需要影像時，再回傳最新的影像。

        return self.Frame
        
    def queryframe(self):
        height, width = 128, 171
        scale = 136
        while (not self.isstop):
            self.status, self.Frame = self.capture.read()
            self.img = self.Frame
            if len(self.Framecube) < 30:
                if self.x % 2==0:

                    self.img = cv2.resize(self.img, (width, height), interpolation = cv2.INTER_LINEAR)
                    off_h = abs(np.ceil((height - scale) / 2).astype(int))
                    off_w = abs(np.ceil((width - scale) / 2).astype(int))
                    self.img = self.img[off_h:off_h + scale, off_w:off_w + scale, :]
                    self.img = cv2.resize(self.img, (112, 112), interpolation = cv2.INTER_LINEAR)

                    self.Framecube.append(self.img)
                    # self.Framecube.append(self.Frame)
                    if self.x >= 1800:
                        self.x = 0
                        self.x += 1
                    else:
                        self.x+=1     
                else:
                    self.x+=1
                
            
            elif len(self.Framecube) >= 30 and len(self.Framecube) < 180:
                if self.x % 2==0:

                    self.img = cv2.resize(self.img, (width, height), interpolation = cv2.INTER_LINEAR)
                    off_h = abs(np.ceil((height - scale) / 2).astype(int))
                    off_w = abs(np.ceil((width - scale) / 2).astype(int))
                    self.img = self.img[off_h:off_h + scale, off_w:off_w + scale, :]
                    self.img = cv2.resize(self.img, (112, 112), interpolation = cv2.INTER_LINEAR)
                    
                    self.Framecube.append(self.img)
                    # self.Framecube.append(self.Frame)
                    if self.x >= 1800:
                        self.x = 0
                        self.x += 1
                    else:
                        self.x+=1     
                else:
                    self.x+=1
                
                if self.flag:
                    del self.Framecube[0:8]
                    # del self.Framecube[0:10]
                    # del self.Framecube[0:16]
                    self.flag = False
                else:
                    pass
                
        
        self.capture.release()
    def getcubeframe(self):
        if len(self.Framecube) > 30:
            FC = self.Framecube[0:16]
            self.flag = True
            return np.array(FC), str(self.x), len(self.Framecube)
        else:
            return "Preparing", str(self.x), len(self.Framecube)

    def __del__(self):
        print('CAM Capture DEL!')

In [5]:
class GesturePredict:
    def __init__(self, mA, mB, ipcam):

        self.isstop = False
        self.modelA = load_model(str(mA))
        time.sleep(0.03)
        self.modelB = load_model(str(mB))
        self.cubelen = 0
        self.cube = []
        self.predA = []
        self.predB = []
        self.ipcam = ipcam
        self.gestureA = ''
        # self.gestureB = ''
        self.gesturePA = 0.0
        self.gesturePB = 0.0

    def start(self):
        print('Predict Started!')
        threading.Thread(target=self.predict, daemon=True, args=()).start()
    
    def stop(self):
        self.isstop = True
        del self.modelA 
        del self.modelB 
        del self.cube 
        del self.cubelen
        del self.predA 
        del self.predB 
        del self.ipcam 
        del self.gestureA
        # del self.gestureB
        del self.gesturePA
        del self.gesturePB
        print('Predict Stopped!')


    def predict(self):
        predstart = time.time()
        while (not self.isstop):
            time.sleep(0.03)
            C, s, l = self.ipcam.getcubeframe()
            if type(C) == str:
                pass
            else:
                self.cube = C
                self.cubelen = l

            if self.cubelen >= 16:
                # print('CUBE')
                X = self.cube[np.newaxis,:]
                self.predA = self.modelA.predict(X.astype('float32'))
                self.predB = self.modelB.predict(X.astype('float32'))
                now = time.time()

                if not ((self.predA[0] < 0.65).all()) or not((self.predB[0] < 0.65).all()):
                    
                    if np.argmax(self.predA, axis=1)[0] == np.argmax(self.predB, axis=1)[0] and (np.max(self.predA[0]) > 0.84 and np.max(self.predB[0]) > 0.84) and now-predstart > 2.5:
                        predstart = time.time()
                        self.gestureA = str(np.argmax(self.predA, axis=1)[0])
                        self.gesturePA = np.max(self.predA[0])
                        self.gesturePB = np.max(self.predB[0])
                        
                        # if np.max(self.predA[0]) > 0.75 and np.max(self.predB[0]) > 0.75:
                        #     predstart = time.time()
                        #     self.gestureA = str(np.argmax(self.predA, axis=1)[0])
                        #     self.gesturePA = np.max(self.predA[0])
                        #     self.gesturePB = np.max(self.predB[0])
                        # else:
                        #     self.gestureA = ''

                    else:
                        self.gestureA = ''
                    
                else:
                    self.gestureA = ''               

                # if (self.predA[0] < 0.65).all() or (self.predB[0] < 0.65).all():
                #     self.gestureA = ''
                #     # self.gestureB = ''
                # else:
                #     if np.argmax(self.predA, axis=1)[0] == np.argmax(self.predB, axis=1)[0] and now-predstart > 2.5:

                #         if np.max(self.predA[0]) < 0.73 or np.max(self.predB[0]) < 0.73:
                #             self.gestureA = ''
                #             # self.gestureB = ''
                #         else:
                #             # print(now-predstart)
                #             predstart = time.time()
                        
                #             self.gestureA = str(np.argmax(self.predA, axis=1)[0])
                #             # self.gestureB = str(np.argmax(self.predB, axis=1)[0])

                #             self.gesturePA = np.max(self.predA[0])
                #             self.gesturePB = np.max(self.predB[0])
                #             # print(self.gesturePA,self.gesturePB)                  

            else:
                pass

                      
        self.ipcam.stop()

    def getgesture(self):

        AGA = self.gestureA
        # BGB = self.gestureB
        PGA = self.gesturePA
        PGB = self.gesturePB
        self.gestureA = ''
        self.gestureB = ''
        self.gesturePA = 0.0
        self.gesturePB = 0.0

        # return AGA, BGB, PGA, PGB
        return AGA, PGA, PGB

    def __del__(self):
        print('Model Predict DEL!')

    

In [6]:
# class GesturePredict:
#     def __init__(self, mA, ipcam):

#         self.isstop = False
#         self.modelA = load_model(str(mA))
#         self.cubelen = 0
#         self.cube = []
#         self.predA = []
#         self.ipcam = ipcam
#         self.gestureA = ''
#         self.gesturePA = 0.0

#     def start(self):
#         print('Predict Started!')
#         threading.Thread(target=self.predict, daemon=True, args=()).start()
    
#     def stop(self):
#         self.isstop = True
#         del self.modelA  
#         del self.cube 
#         del self.cubelen
#         del self.predA 
#         del self.ipcam 
#         del self.gestureA
#         del self.gesturePA
#         print('Predict Stopped!')


#     def predict(self):
#         predstart = time.time()
#         while (not self.isstop):
#             time.sleep(0.03)
#             C, s, l = self.ipcam.getcubeframe()
#             if type(C) == str:
#                 pass
#             else:
#                 self.cube = C
#                 self.cubelen = l

#             if self.cubelen >= 16:
#                 # print('CUBE')
#                 X = self.cube[np.newaxis,:]
#                 self.predA = self.modelA.predict(X.astype('float32'))
#                 now = time.time()
                

#                 if not(self.predA[0] < 0.65).all():

#                     if  np.max(self.predA[0]) > 0.88 and now-predstart > 2.5:
                        
#                         predstart = time.time()
#                         self.gestureA = str(np.argmax(self.predA, axis=1)[0])
#                         self.gesturePA = np.max(self.predA[0])

#                     else:
#                         self.gestureA = ''
                    
#                 else:
#                     self.gestureA = ''                                           

#             else:
#                 pass

                      
#         self.ipcam.stop()

#     def getgesture(self):

#         AGA = self.gestureA
#         PGA = self.gesturePA
#         self.gestureA = ''
#         self.gesturePA = 0.0

#         return AGA, PGAddadws[]dwadwsaswsadws][][adㄊ]

#     def __del__(self):
#         print('Model Predict DEL!')

In [7]:
ipcam = ipcamCapture(int(1))
# ipcam = ipcamCapture(0)

# GSP = GesturePredict(A, ipcam)
GSP = GesturePredict(A,B,ipcam)

predstart = time.time()

DetectFlag = False
showcount = 0
Gesture = ''

# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
  
# org 
org = (50, 50) 
  
# fontScale 
fontScale = 1.3
   
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2


Gestures = [
           'Swiping Left',
           'Swiping Right', 
           'Swiping Down', 
           'Swiping Up', 
           'Turning Hand Clockwise', 
           'Turning Hand Counterclockwise'
           ]

# Web CAM Start
ipcam.start()

# Model Predict Start
GSP.start()

# 暫停1秒，確保影像已經填充
time.sleep(1)

# 使用無窮迴圈擷取影像，直到按下Esc鍵結束
while True:
    I = ipcam.getframe()
    # aGA, pGA = GSP.getgesture()
    aGA, pGA, pGB = GSP.getgesture()
    # aGA, aGB, pGA, pGB = GSP.getgesture()
    time.sleep(0.05)

    if not(aGA=='') or not(pGA == 0.0):
        # print(aGA,pGA)
        print(aGA,pGA,pGB)
        # print(aGA,aGB,pGA,pGB)
        Gesture = Gestures[int(aGA)]
        KBC(int(aGA))
        DetectFlag = True
        showcount = 0
        
    else:
        pass       

    if showcount >= 15:
        DetectFlag = False
        showcount = 0
    else:
        pass
        
    if DetectFlag:
        showcount+=1
        I = cv2.putText(I, Gesture, org, font,  fontScale, color, thickness, cv2.LINE_AA) 
    else:
        showcount = 0
    
    
    cv2.imshow('Image', I)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        GSP.stop()
        ipcam.stop()
        del ipcam
        del GSP
        break

ipcam started!
Predict Started!
0 0.9906371 0.93569773
1 0.9239225 0.9292676
0 0.9753872 0.98836005
4 0.9768908 0.9113167
4 0.9926892 0.91201556
3 0.9226725 0.96008945
0 0.9798006 0.99672455
1 0.952845 0.9691311
5 0.9901106 0.93193096
4 0.9528112 0.9829275
2 0.97242635 0.9893482
3 0.9981375 0.98184973
3 0.99961483 0.99691904
0 0.9993668 0.9993007
1 0.98863226 0.9971641
5 0.9191737 0.87792015
4 0.94286925 0.9338798
4 0.9863036 0.9938763
0 0.99654895 0.99863714
1 0.8699309 0.9099899
3 0.9594765 0.97449523
2 0.9994149 0.9991285
4 0.9096509 0.9753935
4 0.85942054 0.9729451
5 0.9962715 0.88643974
4 0.9890932 0.99528164
0 0.94549954 0.9668519
1 0.9978635 0.9965055
1 0.9323772 0.9938876
3 0.9534766 0.9336183
2 0.9996736 0.9983917
1 0.86686397 0.9209908
1 0.9806045 0.99168277
1 0.9984055 0.9944412
0 0.9983322 0.9883646
1 0.9669075 0.99976104
0 0.97312003 0.9933286
0 0.92541957 0.9891366
3 0.9590501 0.9934257
2 0.9885512 0.9346864
0 0.9600355 0.9766383
0 0.9571075 0.98840827
1 0.99876595 0.9985

In [8]:
# GSP.stop()
# del GSP

In [9]:
# ipcam.stop()